In [1]:
import xml.etree.ElementTree as ET
from xml.dom import minidom
from bs4 import BeautifulSoup
import re
import nltk
import gensim
from nltk.tokenize import RegexpTokenizer
from nltk.corpus import stopwords
nltk.download('stopwords')
from nltk.tokenize import word_tokenize

tokenizer = RegexpTokenizer(r'\w+')

[nltk_data] Downloading package stopwords to /home/rodjo/nltk_data...
[nltk_data]   Package stopwords is already up-to-date!


In [2]:
# load XML file 
xmlTree = ET.parse('Posts.xml')
root = xmlTree.getroot()

tags = {elem.tag for elem in xmlTree.iter()}
xml_document = minidom.parse('Posts.xml')
#xml_document = minidom.parse('/content/Posts.xml')
row_elements = xml_document.getElementsByTagName('row')
posts_elements = xml_document.getElementsByTagName('posts')

#print("The amount of posts elements: ", len(posts_elements))
#print("The amount of row elements: ", len(row_elements))

In [3]:
# This cell contains the prints which describe the structure of root element
# the text is in a body attribute of a root elemnt

row_element = root[0]
print("The content of arrtibutes segment of row element: ")
print(row_element.attrib)
print("")


The content of arrtibutes segment of row element: 
{'Id': '1', 'PostTypeId': '1', 'AcceptedAnswerId': '3', 'CreationDate': '2010-09-13T19:27:46.227', 'Score': '15', 'ViewCount': '253', 'Body': '<p>Are questions that belong to the category "What Apps Should I Get?" appropriate Android Enthusiasts? How about if they\'re asking for specific apps, such as "What are good RSS aggregator apps that integrate with Google Reader and allow offline reading?"</p>\n', 'OwnerUserId': '36', 'LastEditorUserId': '440', 'LastEditDate': '2013-06-05T06:10:40.070', 'LastActivityDate': '2013-06-05T06:10:40.070', 'Title': 'Are "What Apps Should I Get?" Questions Appropriate?', 'Tags': '<discussion><faq><scope><questions>', 'AnswerCount': '10', 'CommentCount': '2', 'ContentLicense': 'CC BY-SA 2.5'}



In [4]:
string = row_element.attrib['Body']
cleantext = BeautifulSoup(string, features="html.parser").text
# print example of old and new text
print(cleantext)

Are questions that belong to the category "What Apps Should I Get?" appropriate Android Enthusiasts? How about if they're asking for specific apps, such as "What are good RSS aggregator apps that integrate with Google Reader and allow offline reading?"



In [5]:
list = []

for string in root.iter('row'):
    body = string.attrib['Body']
    cleantext = BeautifulSoup(body, features="html.parser").text
    list.append(cleantext)
    

/usr/local/lib/python3.10/dist-packages/bs4/__init__.py:435: MarkupResemblesLocatorWarning: The input looks more like a filename than markup. You may want to open this file and pass the filehandle into Beautiful Soup.
  warnings.warn(


In [6]:
# now we need to delete hyperlinks, both http and https
for i in range(len(list)):
    list[i] = re.sub(r'http\S+', '', list[i])     

# remove new lines    
for i in range(len(list)):
    list[i].replace("\n", "")    
    
# apply tockenizer to split words to elements    
for i in range(len(list)):
    word_lower = list[i].lower()
    cleaned_word = tokenizer.tokenize(word_lower)
    list[i] = cleaned_word
    

In [ ]:
# track the current progress
progess = 0

# Less documents, less execution time
list = list[:100]

for i in range(len(list)):
    list[i] = [word for word in list[i] if not word in stopwords.words()]
    progess += 1
    if progess % 100 == 0: 
        print(progess)


In [ ]:
model = gensim.models.Word2Vec(list, min_count = 1, window = 5, vector_size=20)   # newer gensim versions, vector_size == size
#model = gensim.models.Word2Vec(list, min_count = 1, window = 5, size=20)   


In [ ]:
dictionary = dict({})
#for idx, key in enumerate(model.wv.vocab):  #< 4 version
for idx, key in enumerate(model.wv.key_to_index):
    dictionary[key] = model.wv[key]

In [ ]:
print(type(dictionary))

In [ ]:
"""
The method shoud expand the currenty query based on the cosine similartiy based on the given threshold. 
The method should return the list of elements, including the expending ones (if any). 

Parameters
----------
dictionary : dictionary
    Dictionary based on the vec2word model 
original_query : str
    The query
number_of_elements_to_expand: int
    Number of elements to expand
threshold : float, optional
    The similarity threshold required to add word to a query (default = 0.00)
"""
from sklearn.metrics.pairwise import cosine_similarity   # add import to beginning
import numpy as np

def cossim(query_vectors, doc_vectors):
    similarities = []
    for query_vector in query_vectors:
        query_sim = []
        for doc_vector in doc_vectors:
            similarity = cosine_similarity(query_vector.reshape(1, -1), doc_vector.reshape(1, -1))
            #print(similarity[0][0])
            query_sim.append(similarity[0][0])
        similarities.append(query_sim)

    return similarities

def queryExpansionAlgorithm(dictionary, original_query, number_of_elements_to_expand, threshold=0.00):
    query_tokens = original_query.split()
    query_vectors = [model.wv[token] for token in query_tokens]
    # 1) add preprocessing steps in code above to handle tokens that are not in our current model. OR handle with different idea
    # 2) add error handling in case token is not in model -> ignore this part of query for example = remove token
    #print(query_vectors)
    #print(dictionary.values())
    similarity_matrix = cossim(query_vectors, dictionary.values())

    dict_keys_list = []
    for key in dictionary.keys():
        dict_keys_list.append(key)

    expanded_query_elements = []
    for i, query_word in enumerate(query_tokens):
        # top n highest similarity
        top_n = np.argpartition(similarity_matrix[i], -number_of_elements_to_expand)[-number_of_elements_to_expand:]
        #rint(top_n)

        # Check if the cosine similarity between each of the top N words and the original word is above the threshold
        for j in top_n:
            if similarity_matrix[i][j] >= threshold:
                expanded_query_elements.append((dict_keys_list[j], similarity_matrix[i][j]))
    
    return expanded_query_elements
    
    
print(queryExpansionAlgorithm(dictionary, "code developers", 4))